In [4]:
sc

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-4>

In [5]:
from pyspark.sql import functions as func
#Specifying the schema instead of inferring it 
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType, DoubleType, LongType

fileSchema = StructType([StructField('movie_id', IntegerType(),True),
                        StructField('user_id', IntegerType(),True),
                        StructField('rating', IntegerType(),True),
                        StructField('time_stamp', LongType(),True)])

movie_ratings = spark.read.load("s3://sparkbucket123/u.data", format="csv", sep="\t", schema = fileSchema, header="true")
movie_ratings= movie_ratings.repartition("movie_id")
movie_ratings_cache = movie_ratings.cache()
movie_ratings_cache.show();

movie_rating_count = movie_ratings_cache.groupBy('movie_id').agg(func.count("rating"))

movie_rating_count.show()


#Reading the movie id and movie name
movies = spark.read.load("s3://sparkbucket123/u.item", format="csv", sep="|", inferSchema="true", header="true")
movie_selected = movies.select('10', 'Toy Story (1995)')
from functools import reduce
oldColumns = movie_selected.schema.names
newColumns = ["movie_id", "movie_name"]
movie_names = reduce(lambda movie_selected, idx: movie_selected.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), movie_selected)
movie_names = movie_names.repartition("movie_id")
movie_names_cache = movie_names.cache()
movie_names_cache.show()


#Joining the two movies data
joined_movie_data = movie_ratings_cache.join(movie_names.hint("broadcast") , ["movie_id"])
joined_movie_data.show()


#Finding which movie name got how many ratings
movie_name_ratings_count = joined_movie_data.groupBy('movie_name').agg(func.count("rating"))
movie_name_ratings_count.show()



#Saving this data in the disk to be used later
movie_name_ratings_count.write.orc("movie_ratings2.orc")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+------+----------+
|movie_id|user_id|rating|time_stamp|
+--------+-------+------+----------+
|     148|    408|     5| 877399018|
|     148|      1|     4| 877019411|
|     148|    140|     1| 877019882|
|     148|    114|     5| 877016735|
|     148|     70|     5| 877021271|
|     148|    495|     4| 877016735|
|     148|    172|     5| 877016513|
|     148|   1149|     5| 877016513|
|     148|    177|     2| 877020715|
|     148|    185|     1| 877398385|
|     148|    501|     4| 877020297|
|     148|    529|     5| 877398901|
|     148|     98|     3| 877017714|
|     148|    214|     5| 877019882|
|     148|    596|     5| 877020297|
|     148|     71|     5| 877019251|
|     148|   1012|     4| 877400154|
|     148|    228|     4| 877016514|
|     148|    473|     5| 877399322|
|     148|    357|     5| 877016735|
+--------+-------+------+----------+
only showing top 20 rows

+--------+-------------+
|movie_id|count(rating)|
+--------+-------------+
|     148| 

In [6]:
#Reading it later
movie_ratings_stored=spark.read.orc("movie_ratings2.orc")
movie_ratings_stored.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------+
|          movie_name|count(rating)|
+--------------------+-------------+
|Sword in the Ston...|          106|
|Star Trek IV: The...|          133|
|Full Monty, The (...|          323|
|Hellraiser: Blood...|           48|
|Arsenic and Old L...|          191|
|       Brazil (1985)|           38|
|In the Mouth of M...|           28|
|Year of the Horse...|           43|
|3 Ninjas: High No...|          245|
|For Richer or Poo...|           59|
|Garden of Finzi-C...|          107|
|    Boot, Das (1981)|           40|
|In the Bleak Midw...|           39|
|Singin' in the Ra...|          114|
|Alice in Wonderla...|           36|
|     Spy Hard (1996)|          226|
|    True Lies (1994)|          281|
|Down Periscope (1...|           20|
|         Jack (1996)|          203|
| Exit to Eden (1994)|           25|
+--------------------+-------------+
only showing top 20 rows